# Práctica sobre algoritmos genéticos

Un área ferroviaria de carga/descarga con una única vía de entrada y otra salida se compone de tres muelles de carga/descarga: Op1, Op2 y Op3, correspondientes a contenedores, carbón y gas. Por tanto, cada tren que llega se dirige a un muelle en función de su carga. Un tren tarda en cargar o descargar un tiempo proporcional al número de vagones que arrastra. Cada día llegan secuencialmente n trenes. Si los trenes son de cargas distintas, pueden entrar en paralelo a los muelles. Cuando dos trenes con el mismo tipo de carga se encuentran seguidos, el segundo debe esperar por el primero, así como todos los demás que se encuentren por detrás.

Se nos plantea resolver, mediante un algoritmo genético, el problema de la ordenación en la entrada de los trenes para minimizar el tiempo de paso del conjunto de trenes.


In [ ]:
import random
import numpy
import matplotlib.pyplot as plt
from deap import base, creator, tools, algorithms

## Trenes

Se define la clase Train, que representa un tren y sus atributos, como el número de vagones, el muelle de operaciones y la matrícula.

In [ ]:
class Train:
    def __init__(self, wagons, op, licence_plate):
        self.wagons = wagons
        self.op = op
        self.licence_plate = licence_plate

    def __str__(self):
        return "Número de vagones:" + str(self.wagons) \
        + "\n" + "Muelle de operaciones:" + str(self.op) \
        + "\n" + "Matrícula:" + str(self.licence_plate)


La función random_trains_generation genera una lista de trenes de manera aleatoria. Cada tren tiene un número aleatorio de vagones y se le asigna un tipo de carga (op1, op2 o op3).

In [ ]:
def random_trains_generation(n):

    train_list = []
    
    for i in range(n):
        wagons = random.randint(10, 30)  # Cada tren puede arrastrar entre 10 y 30 vagones
        op = random.choice(["op1", "op2", "op3"])  # A cada tren se le asigna un tipo de carga
        train_list.append(Train(wagons, op, i))

    return train_list

trains = random_trains_generation(100)

Se crea el tipo de fitness FitnessMin con el peso establecido como -1.0, lo que indica que se busca minimizar el valor del fitness. Además, se crea el tipo de individuo Individual como una lista que contiene objetos de la clase Train y se le asigna el tipo de fitness definido anteriormente.

In [ ]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

Se crea una instancia del Toolbox de DEAP y se registran las funciones y operadores genéticos necesarios, como la inicialización de individuos, la inicialización de la población, la evaluación de individuos, la selección, la cruza y la mutación

In [ ]:
toolbox = base.Toolbox()
toolbox.register("individual", creator.Individual, trains)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

La función evalTrains calcula el tiempo total de paso de los trenes en un individuo. Se utiliza un diccionario use_times para realizar un seguimiento de los tiempos de uso de los muelles de carga/descarga. Se itera sobre los trenes del individuo y se actualizan los tiempos de uso de los muelles según las reglas establecidas. Al final, se calcula el tiempo total sumando los tiempos de espera y el tiempo máximo de uso de los muelles.

In [ ]:
def evalTrains(individual):
    use_times = {"op1": 0, "op2": 0, "op3": 0}
    total_time = 0

    for train in individual:
        if use_times[train.op] == 0:
            use_times[train.op] = train.wagons
        else:
            waiting_time = use_times[train.op]
            for op, value in use_times.items():
                if value - waiting_time < 0:
                    use_times[op] = 0
                else:
                    use_times[op] = value - waiting_time
            use_times[train.op] = train.wagons
            total_time += waiting_time

    total_time += max(use_times.values())
    return total_time,

Se crea un conjunto licence_plates que contiene todas las matrículas de los trenes generados. Esto se utiliza más adelante para verificar qué trenes faltan en un individuo.

In [ ]:
licence_plates = set()
for t in trains:
    licence_plates.add(t.licence_plate)

Las funciones get_missing_trains y remove_repeated se utilizan para obtener los trenes que faltan en un individuo y eliminar los trenes repetidos, respectivamente.

In [ ]:
def get_missing_trains(ind):
    missing_ind = licence_plates - set(train.licence_plate for train in ind)
    return [t for t in trains if t.licence_plate in missing_ind]

def remove_repeated(ind):
    return [t for i, t in enumerate(ind) if t.licence_plate not in {x.licence_plate for x in ind[:i]}]

La función cxOnePointModified es una versión modificada del operador de cruza de un punto. Se selecciona un punto de cruza aleatorio y se intercambian los segmentos seleccionados entre los padres. Luego, se asegura que no haya trenes repetidos ni trenes faltantes en los descendientes utilizando las funciones auxiliares mencionadas anteriormente.

In [ ]:
def cxOnePointModified(ind1, ind2):
    cxpoint = random.randint(0,len(ind1) - 1)

    # Intercambiar los segmentos seleccionados entre los padres
    ind1[cxpoint:], ind2[cxpoint:] = ind2[cxpoint:], ind1[cxpoint:]

    missing_ind1 = get_missing_trains(ind1)
    missing_ind2 = get_missing_trains(ind2)
    none_repeated_ind1 = remove_repeated(ind1)
    none_repeated_ind2 = remove_repeated(ind2)
    
    ind1[:] = none_repeated_ind1 + missing_ind1
    
    ind2[:] = none_repeated_ind2 + missing_ind2

    return ind1, ind2

La función main es la función principal del algoritmo genético. Se establecen los parámetros como el número de generaciones, el tamaño de la población, las probabilidades de cruza y mutación, y se inicializa la población. También se definen las estadísticas y el "Hall of Fame" para almacenar los mejores individuos. Luego, se llama a la función eaSimple de DEAP para ejecutar el algoritmo genético con los operadores y parámetros especificados.

In [ ]:
toolbox.register("evaluate", evalTrains)
toolbox.register("mate", cxOnePointModified)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=4)

def main():
    NGEN = 500
    MU = 10
    CXPB = 0.7
    MUTPB = 0.2

    # Población inicial
    pop = toolbox.population(n=MU)
    
    # Estadísticas de la población
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean, axis=0)
    stats.register("min", numpy.min, axis=0)
    stats.register("max", numpy.max, axis=0)

    # Hall Of Fame
    hof = tools.HallOfFame(4)


    algorithms.eaSimple(pop, toolbox, cxpb=CXPB, mutpb=MUTPB, ngen=NGEN, stats=stats, halloffame=hof, verbose=True)

    return pop, stats, hof

In [ ]:
pop, stats, hof  = main()

### Tiempo Ideal vs Tiempo del algoritmo

Esta sección calcula el tiempo mínimo esperado para completar las operaciones y compara este valor con el tiempo obtenido por el algoritmo genético. Se realiza un seguimiento del tiempo requerido para cada operación y se imprime el tiempo mínimo y el tiempo del algoritmo.

In [ ]:
time_of_operations = {"op1": 0, "op2": 0, "op3": 0}
for t in trains:
    time_of_operations[t.op] += t.wagons
print("el tiempo mínimo será: ",max(time_of_operations["op1"], time_of_operations["op2"], time_of_operations["op3"]))
print("el tiempo del algoritmo será: ",hof[0].fitness.values[0])

### Revisión de resultados repetidos

Aquí se verifica si existen resultados duplicados en la mejor solución encontrada por el algoritmo genético. Se crea un conjunto de placas de licencia únicas para eliminar duplicados y se imprime la longitud de la mejor solución, el conjunto de placas de licencia únicas y la lista completa de placas de licencia.

In [ ]:
licence_plates = set()
licencias = []
for t in hof[0]:
    licence_plates.add(t.licence_plate)
    licencias.append(t.licence_plate)

print(len(hof[0]))
print(licence_plates)
print(licencias)